In [3]:
import requests
import pandas as pd
import numpy as np
import pyodbc
import psycopg2
from datetime import datetime
from dotenv import load_dotenv
import os


ModuleNotFoundError: No module named 'dotenv'

In [17]:
# Carrega as variáveis de ambiente do arquivo .env
load_dotenv(dotenv_path=r"C:\Users\Matheus Alexander\Desktop\DW\Data-Warehouse\Environment\.env")

True

In [20]:
# Acesse as variáveis de ambiente
server = os.getenv('SERVER_MIS')
database = os.getenv('DATABASE_MIS')
username = os.getenv('USERNAME_MIS')
password = os.getenv('PASSWORD_MIS')

# Verifica se as variáveis foram carregadas com sucesso
if server is None or database is None or username is None or password is None:
    print("Erro: Não foi possível carregar todas as variáveis de ambiente.")
    # Aqui você pode decidir o que fazer em caso de erro, como lançar uma exceção ou encerrar o programa
else:
    print("Variáveis de ambiente carregadas com sucesso.")


Variáveis de ambiente carregadas com sucesso.
